# MCP demo – Test (CLI + chiamate tool)

Questo notebook ti fa vedere:
- che i file esistono
- come lanciare il client CLI
- come chiamare direttamente i tool MCP da Python (senza CLI)

Assicurati di aver installato le dipendenze nel tuo ambiente Python attivo in VS Code.


## 1.1 – Verifica file progetto

In [ ]:

from pathlib import Path
ROOT = Path("mcp-demo").resolve()
for p in [
    ROOT/"servers/arxiv_server.py",
    ROOT/"servers/openalex_server.py",
    ROOT/"cli/mcp_chat.py",
    ROOT/".vscode/mcp.json",
]:
    print(p, "->", "OK" if p.exists() else "MISSING")


## 1.2 – Esegui il client CLI (da notebook)

Puoi eseguirlo anche da terminale VS Code:

```bash
cd mcp-demo
python cli/mcp_chat.py "agentic RAG evaluation" --k 3
```

Qui sotto lo lanciamo direttamente dal notebook (usa lo stesso Python del kernel).


In [ ]:

import sys, subprocess, shlex
from pathlib import Path

cmd = [sys.executable, "cli/mcp_chat.py", "agentic RAG evaluation", "--k", "3"]
print("Running:", " ".join(shlex.quote(c) for c in cmd))

subprocess.run(cmd, cwd=Path("mcp-demo"), check=True)


## 1.3 – Chiamate tool “a mano” (senza CLI)

Qui facciamo:
- avvio server arXiv e OpenAlex via stdio
- discovery tools
- chiamata tool e stampa risultati in Python


In [ ]:

import asyncio
from fastmcp import Client
from fastmcp.client.transports import StdioTransport

async def run():
    arxiv = Client(StdioTransport(command="python", args=["servers/arxiv_server.py"]))
    openalex = Client(StdioTransport(command="python", args=["servers/openalex_server.py"]))

    async with arxiv, openalex:
        tools_a = await arxiv.list_tools()
        tools_o = await openalex.list_tools()
        print("Tools arXiv:", [t.name for t in tools_a])
        print("Tools OpenAlex:", [t.name for t in tools_o])

        ax = await arxiv.call_tool("arxiv_search", {"topic":"retrieval augmented generation", "max_results":2})
        ox = await openalex.call_tool("openalex_search_works", {"query":"retrieval augmented generation", "per_page":2})

        return ax.data, ox.data

ax_data, ox_data = asyncio.run(run())
ax_data[0]["title"], ox_data[0]["title"]


## 1.4 – (Opzionale) visualizza i risultati in tabella

In [ ]:

import pandas as pd

df_ax = pd.DataFrame(ax_data)[["title","published","pdf_url"]]
df_ox = pd.DataFrame(ox_data)[["title","publication_year","cited_by_count","openalex_url"]]

display(df_ax)
display(df_ox)


In [1]:
from fastmcp import Client
from fastmcp.client.transports import StdioTransport

openalex = Client(StdioTransport(command="python", args=["servers/openalex_server.py"]))

async with openalex:
    tools = await openalex.list_tools()
    print("TOOLS:", [t.name for t in tools])

    res = await openalex.call_tool(
        "openalex_search_works",
        {"query": "agentic RAG evaluation", "per_page": 3, "mode": "general"}
    )

print("RAW RES:", res)
print("UNPACK:", unpack(res) if "unpack" in globals() else res)


TOOLS: ['openalex_search_works']
RAW RES: CallToolResult(content=[TextContent(type='text', text='{\n  "id": "https://openalex.org/W4290998907",\n  "doi": "10.1097/00000478-198911000-00001",\n  "title": "Bacillary Angiomatosis",\n  "publication_year": 1989,\n  "cited_by_count": 220,\n  "primary_location": "The American Journal of Surgical Pathology",\n  "openalex_url": "https://openalex.org/W4290998907"\n}', annotations=None, meta=None), TextContent(type='text', text='{\n  "id": "https://openalex.org/W2067971687",\n  "doi": "10.1021/tx300455k",\n  "title": "Cadmium-Based Quantum Dot Induced Autophagy Formation for Cell Survival via Oxidative Stress",\n  "publication_year": 2013,\n  "cited_by_count": 134,\n  "primary_location": "Chemical Research in Toxicology",\n  "openalex_url": "https://openalex.org/W2067971687"\n}', annotations=None, meta=None), TextContent(type='text', text='{\n  "id": "https://openalex.org/W4412708385",\n  "doi": "10.1136/bmjhci-2025-101570",\n  "title": "Developme